In [9]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import plotly.express as px

pd.set_option('display.max_columns', 500)  # Display all columns
pd.set_option('display.max_rows', 500)     # Display all rows


In [10]:
df = pd.read_csv("../data/data.csv")

In [11]:
df.head()

,author,author_email,creator_user_id,id,isopen,license_id,license_title,maintainer,maintainer_email,metadata_created,metadata_modified,name,notes,num_resources,num_tags,organization,owner_org,private,state,title,type,url,version,extras,groups,resources,tags,relationships_as_subject,relationships_as_object,license_url
0,NaN,NaN,1ecd1fb1-1be6-46bb-b90d-07a0762ed104,7401571a-9009-44d1-96ad-b7003221ec72,False,notspecified,License not specified,Western Pennsylvania Regional Data Center,wprdc@pitt.edu,2023-01-24T18:11:53.465876,2023-01-24T18:11:53.465881,pittsburgh-american-community-survey-2015-misc...,Miscellaneous Reports from the 2015 American C...,20,19,"{'id': 'e846364e-cb2f-4a9a-bcee-1f4f570d10e3',...",e846364e-cb2f-4a9a-bcee-1f4f570d10e3,False,active,Pittsburgh American Community Survey 2015 - Mi...,dataset,NaN,NaN,"[{'key': 'resource-type', 'value': 'Dataset'},...",[{'description': 'Local Government Topic - for...,"[{'cache_last_updated': None, 'cache_url': Non...","[{'display_name': 'acs', 'id': 'a1da5efa-ee43-...",[],[],NaN
1,NaN,NaN,1ecd1fb1-1be6-46bb-b90d-07a0762ed104,26bae39e-d740-411c-b078-8f5d088dcbb1,False,NaN,NaN,NaN,NaN,2022-11-01T19:50:22.184970,2022-11-01T19:50:22.184978,tiger-line-shapefile-2019-nation-u-s-current-m...,The TIGER/Line shapefiles and related database...,4,6,"{'id': 'fb3131aa-ef06-4a00-ad84-67d93a71d7e3',...",fb3131aa-ef06-4a00-ad84-67d93a71d7e3,False,active,"TIGER/Line Shapefile, 2019, nation, U.S., Cur...",dataset,NaN,NaN,"[{'key': 'guid', 'value': 'tl_2019_us_cbsa.shp...",[],"[{'cache_last_updated': None, 'cache_url': Non...","[{'display_name': 'cbsa', 'id': '54a64943-033c...",[],[],NaN
2,NaN,NaN,2b785922-9f13-491b-a3c2-2a40acbd80c2,69ec8980-70b3-4713-b623-a57e135f899d,False,notspecified,License not specified,NYC OpenData,no-reply@data.cityofnewyork.us,2020-11-10T17:03:50.599068,2024-02-02T16:22:54.277770,citywide-payroll-data-fiscal-year,Data is collected because of public interest i...,4,0,"{'id': '1149ee63-2fff-494e-82e5-9aace9d3b3bf',...",1149ee63-2fff-494e-82e5-9aace9d3b3bf,False,active,Citywide Payroll Data (Fiscal Year),dataset,NaN,NaN,"[{'key': 'resource-type', 'value': 'Dataset'},...",[{'description': 'Local Government Topic - for...,"[{'cache_last_updated': None, 'cache_url': Non...",[],[],[],NaN
3,NaN,NaN,2b785922-9f13-491b-a3c2-2a40acbd80c2,fe912bcf-ffdb-4a34-8048-e8277cbc51e4,False,notspecified,License not specified,George Carter,george.r.carter@hud.gov,2020-11-10T16:31:16.666758,2024-03-01T02:10:26.256866,metropolitan-area-look-up,This system provides the user with a facility ...,1,4,"{'id': '7f8ee588-32a3-4b6c-b435-d6603c91dbcc',...",7f8ee588-32a3-4b6c-b435-d6603c91dbcc,False,active,Metropolitan Area Look-Up,dataset,NaN,NaN,"[{'key': 'resource-type', 'value': 'Dataset'},...",[],"[{'cache_last_updated': None, 'cache_url': Non...","[{'display_name': 'cbsa', 'id': '54a64943-033c...",[],[],NaN
4,NaN,NaN,1ecd1fb1-1be6-46bb-b90d-07a0762ed104,6abd8e7a-68b0-48dc-92dc-bc8e199a4314,False,NaN,NaN,NaN,NaN,2022-11-01T20:04:52.551330,2022-11-01T20:04:52.551338,tiger-line-shapefile-2020-nation-u-s-core-base...,The TIGER/Line shapefiles and related database...,4,6,"{'id': 'fb3131aa-ef06-4a00-ad84-67d93a71d7e3',...",fb3131aa-ef06-4a00-ad84-67d93a71d7e3,False,active,"TIGER/Line Shapefile, 2020, Nation, U.S., Core...",dataset,NaN,NaN,"[{'key': 'guid', 'value': 'tl_2020_us_cbsa.shp...",[],"[{'cache_last_updated': None, 'cache_url': Non...","[{'display_name': 'cbsa', 'id': '54a64943-033c...",[],[],NaN


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3225 entries, 0 to 3224
Data columns (total 30 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   author                    0 non-null      float64
 1   author_email              0 non-null      float64
 2   creator_user_id           3225 non-null   object 
 3   id                        3225 non-null   object 
 4   isopen                    3225 non-null   bool   
 5   license_id                2718 non-null   object 
 6   license_title             2718 non-null   object 
 7   maintainer                2713 non-null   object 
 8   maintainer_email          2694 non-null   object 
 9   metadata_created          3225 non-null   object 
 10  metadata_modified         3225 non-null   object 
 11  name                      3225 non-null   object 
 12  notes                     3215 non-null   object 
 13  num_resources             3225 non-null   int64  
 14  num_tags

In [13]:
df['notes'] = df['notes'].fillna('')


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3225 entries, 0 to 3224
Data columns (total 30 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   author                    0 non-null      float64
 1   author_email              0 non-null      float64
 2   creator_user_id           3225 non-null   object 
 3   id                        3225 non-null   object 
 4   isopen                    3225 non-null   bool   
 5   license_id                2718 non-null   object 
 6   license_title             2718 non-null   object 
 7   maintainer                2713 non-null   object 
 8   maintainer_email          2694 non-null   object 
 9   metadata_created          3225 non-null   object 
 10  metadata_modified         3225 non-null   object 
 11  name                      3225 non-null   object 
 12  notes                     3225 non-null   object 
 13  num_resources             3225 non-null   int64  
 14  num_tags

In [21]:
# Step 1: Convert text to numerical features using TF-IDF
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(df['notes'])

# Step 2: Use KMeans for clustering
k = 100
kmeans = KMeans(n_clusters=k, random_state=42)
df['cluster'] = kmeans.fit_predict(X)

# Step 3: Reduce dimensions to 3D for visualization using PCA
pca = PCA(n_components=3)
X_reduced = pca.fit_transform(X.toarray())

# Step 4: Create a DataFrame for the reduced dimensions and clusters
reduced_df = pd.DataFrame(X_reduced, columns=['PCA1', 'PCA2', 'PCA3'])
reduced_df['cluster'] = df['cluster']
reduced_df['name'] = df['name']
reduced_df['title'] = df['title']
reduced_df['id'] = df['id']


# Step 5: Visualize the clusters in 3D using Plotly Express with hover data
fig = px.scatter_3d(
    reduced_df, 
    x='PCA1', 
    y='PCA2', 
    z='PCA3', 
    color='cluster',
    title='3D Embedding of Text Clusters',
    labels={'PCA1': 'PCA 1', 'PCA2': 'PCA 2', 'PCA3': 'PCA 3'},
    hover_data={
        'name': True,          # Show the title on hover
        'title': True # Show additional info on hover
    },
    color_continuous_scale=px.colors.sequential.Viridis
)

fig.show()

In [23]:
# Step 6: Save the reduced_df DataFrame to a CSV file
output_path = '../data/3d_embedding.csv'
reduced_df.to_csv(output_path, index=False)

In [24]:
reduced_df

,PCA1,PCA2,PCA3,cluster,name,title,id
0,-0.069834,0.010321,-0.040011,27,pittsburgh-american-community-survey-2015-misc...,Pittsburgh American Community Survey 2015 - Mi...,7401571a-9009-44d1-96ad-b7003221ec72
1,0.149395,-0.130848,0.140119,89,tiger-line-shapefile-2019-nation-u-s-current-m...,"TIGER/Line Shapefile, 2019, nation, U.S., Cur...",26bae39e-d740-411c-b078-8f5d088dcbb1
2,0.142255,-0.058678,0.071304,27,citywide-payroll-data-fiscal-year,Citywide Payroll Data (Fiscal Year),69ec8980-70b3-4713-b623-a57e135f899d
3,0.012589,-0.036876,0.011488,27,metropolitan-area-look-up,Metropolitan Area Look-Up,fe912bcf-ffdb-4a34-8048-e8277cbc51e4
4,0.149395,-0.130848,0.140119,89,tiger-line-shapefile-2020-nation-u-s-core-base...,"TIGER/Line Shapefile, 2020, Nation, U.S., Core...",6abd8e7a-68b0-48dc-92dc-bc8e199a4314
...,...,...,...,...,...,...,...
3220,-0.017140,-0.012741,-0.040528,27,mapping-data-of-eelgrass-zostera-marina-distri...,Mapping Data of Eelgrass (Zostera marina) Dist...,c4e2c53a-c642-4858-b5ac-cfec87091047
3221,-0.078050,0.047959,-0.063560,27,geographic-information-system-gis-characteriza...,Geographic Information System (GIS) characteri...,8fa90e45-0964-45de-97b0-fa297e1e5f6c
3222,-0.075443,0.033172,-0.087912,27,data-from-biomarkers-associated-with-vaccine-a...,Data from: Biomarkers associated with vaccine-...,3da665be-7015-4240-90ae-1b105bba66a1
3223,-0.020037,-0.020699,-0.049865,27,hmsrp-hawaiian-monk-seal-blood-values-establis...,HMSRP Hawaiian Monk Seal Blood Values (Establi...,66581b9f-6abf-4784-bc00-a3a48949f082
